In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
    !pip install --no-deps unsloth


In [16]:
import os
import json
import pandas as pd
import re
from tqdm import tqdm

In [4]:
import torch
from unsloth import FastLanguageModel

# Define the model details
max_seq_length = 4096
dtype = None  # Autodetect
load_in_4bit = True # Use 4-bit quantization for memory efficiency

# Load the model with your LoRA adapters merged in
# Unsloth's `from_pretrained` can directly load LoRA adapters from the Hub.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Tejas1615/medical-mstrl7b-lora_adapters_final_16b", # 🚀 YOUR MODEL REPO ID
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
print("✅ Successfully loaded your finetuned model from the Hugging Face Hub!")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-15 12:03:17.270949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755259397.513202      60 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755259397.572223      60 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.5: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Unsloth 2025.8.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ Successfully loaded your finetuned model from the Hugging Face Hub!


In [8]:
prompt_template = """<s>[INST] You are an expert medical assistant. Analyze the following multiple-choice question and provide the single best answer.

Question: {question}

Options:
{options}

What is the correct answer? [/INST]"""

def format_inference_prompt(example):
    """
    Formats a single example from your test JSONL files into the Mistral
    instruction format.
    """
    try:
        question = example["question"].strip()
        # The test files have options as a dictionary, so we format them into a string
        options_str = "\n".join([f"{key}. {value}" for key, value in example["options"].items()])
        return prompt_template.format(question=question, options=options_str)
    except Exception as e:
        print(f"Error formatting example: {e}")
        return None

def parse_model_output(generated_text):
    """
    Extracts the single capital letter (A, B, C, etc.) from the model's
    full text response.
    """
    # The model often outputs "A. The answer is..."
    # This regex finds the first capital letter followed by a period.
    match = re.search(r"^\s*([A-Z])\.", generated_text.strip())
    if match:
        return match.group(1)
    
    # Fallback: if the regex fails, take the first character.
    # This handles cases where the model just outputs "A" or "A The answer..."
    if len(generated_text.strip()) > 0:
        return generated_text.strip()[0]
        
    # If all else fails, return a default value (e.g., 'A')
    return 'A'

In [17]:
import pandas as pd

# List of your test files
test_files = [
    '/kaggle/input/godel-data-2/test_easy.jsonl',
    '/kaggle/input/godel-data-2/test_medium.jsonl',
    '/kaggle/input/godel-data-2/test_hard.jsonl',
]

all_predictions = []
model.eval() # Set the model to evaluation mode

# Loop through each test file
for file_path in test_files:
    print(f"\n🚀 Processing file: {os.path.basename(file_path)}")
    
    try:
        with open(file_path, 'r') as f:
            test_examples = [json.loads(line) for line in f]
    except FileNotFoundError:
        print(f"⚠️ File not found: {file_path}. Skipping.")
        continue

    # Generate answers for each example with a progress bar
    for example in tqdm(test_examples, desc=f"Inferencing {os.path.basename(file_path)}"):
        prompt = format_inference_prompt(example)
        if not prompt:
            continue

        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

        with torch.no_grad():
            # --- ACCURACY IMPROVEMENT ---
            # We set do_sample=False to make the output deterministic and choose
            # the most likely answer instead of a creative one.
            outputs = model.generate(
                **inputs,
                max_new_tokens=50, # We only need the first few tokens for the answer
                do_sample=False,   # This is the key for accuracy
                pad_token_id=tokenizer.eos_token_id # Suppress warning
            )
        
        # Decode the full generated text
        full_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        
        # Extract only the answer part after the final [/INST] tag
        answer_part = full_response.split("[/INST]")[-1].strip()
        
        # Parse the letter from the answer
        predicted_letter = parse_model_output(answer_part)
        
        all_predictions.append({
            "id": example["id"],
            "answer": predicted_letter
        })

# Create the submission DataFrame
submission_df = pd.DataFrame(all_predictions)

# Save to submission.csv
submission_df.to_csv("submission.csv", index=False)

print("\n\n" + "="*80)
print("✅ Inference complete! `submission.csv` has been generated.")
print("="*80)
print("\nHere's a preview of your submission file:\n")
print(submission_df.head(10))



🚀 Processing file: test_easy.jsonl


Inferencing test_easy.jsonl: 100%|██████████| 100/100 [01:21<00:00,  1.23it/s]



🚀 Processing file: test_medium.jsonl


Inferencing test_medium.jsonl: 100%|██████████| 100/100 [01:57<00:00,  1.17s/it]



🚀 Processing file: test_hard.jsonl


Inferencing test_hard.jsonl: 100%|██████████| 100/100 [02:13<00:00,  1.34s/it]



✅ Inference complete! `submission.csv` has been generated.

Here's a preview of your submission file:

                                     id answer
0  0396d20b-11aa-4159-8286-a451371a99f0      C
1  6ff719a1-8a31-4f2a-b717-007443c1bd3f      A
2  3ef59327-18e4-4dd8-b482-666afc9a6833      A
3  d73b4d17-0170-4f1a-bafd-6f3a037e022b      A
4  1a7805f1-c0f0-465a-b996-bb5047cb9501      A
5  7d70e718-bef4-4e7a-8781-87fb2a5d52d6      A
6  afea498d-f47c-4f18-b787-ed60e7dfbdc4      A
7  feffb7ba-1e94-4736-8223-154335ab7581      C
8  2c3b5990-1b2a-47e7-ac46-35e4dddff0e3      A
9  ed0e3c88-e875-42a7-b9f9-4b75c492c6d5      A
